## MO444 Project 3 - Reinforcement Learning

* Rodrigo Araújo Marinho Franco - RA: 233569
* Felipe Marinho Tavares - RA: 265680

### Contributions:
* Rodrigo
 * ...
* Felipe
 * ...

# PART I - Evolutionary Model

# PART II - Reinforcement Learning

In [1]:
import time, os
import traceback
import sys
from util import *

import pacman as pm
import layout as l
import textDisplay
import pacmanAgents, ghostAgents

try:
    import boinc
    _BOINC_ENABLED = True
except:
    _BOINC_ENABLED = False

In [2]:
class Environment:
    def __init__(self, layout="mediumClassic"):
        self.layout = l.getLayout(layout)
        self.beQuiet=True
        self.catchExceptions = False
        self.rules = pm.ClassicGameRules(timeout=30)
        self.reset()

    def reset(self):
        self.display = textDisplay.NullGraphics()
        self.pacman = pacmanAgents.GreedyAgent()
        self.ghosts = [ghostAgents.RandomGhost(i+1) for i in range(self.layout.getNumGhosts())]
        self.agents = [pacmanAgents.GreedyAgent()] + [ghostAgents.RandomGhost(i+1) for i in range(self.layout.getNumGhosts())]
        self.game = self.rules.newGame(self.layout, self.pacman, self.ghosts,
                                       self.display, self.beQuiet, self.catchExceptions)

        # inform learning agents of the game start
        for i in range(len(self.agents)):
            agent = self.game.agents[i]
            if not agent:
                self.game.mute(i)
                # this is a null agent, meaning it failed to load
                # the other team wins
                print("Agent %d failed to load" % i, file=sys.stderr)
                self.game.unmute()
                self.game._agentCrash(i, quiet=True)
                return

        self.agentIndex = self.game.startingIndex
        self.numAgents = len( self.game.agents )


    def update_game_state(self, action):
        # Execute the action
        self.game.state = self.game.state.generateSuccessor( self.agentIndex, action )
        # Change the display
        self.game.display.update( self.game.state.data )
        # Allow for game specific conditions (winning, losing, etc.)
        self.game.rules.process(self.game.state, self.game)
        # Track progress
        if self.agentIndex == self.numAgents + 1: self.game.numMoves += 1
        # Next agent
        self.agentIndex = ( self.agentIndex + 1 ) % self.numAgents

        if _BOINC_ENABLED:
            boinc.set_fraction_done(self.game.getProgress())

    def step(self):
        for agent in self.game.agents:
            if not self.game.gameOver:
                observation = self.game.state.deepCopy()
                action = agent.getAction(observation)
                self.update_game_state(action)

    def done(self):
        if not self.game.gameOver:
            return False
        else:
            # inform a learning agent of the game result
            for agentIndex, agent in enumerate(self.game.agents):
                if "final" in dir( agent ) :
                    try:
                        self.game.mute(agentIndex)
                        agent.final( self.game.state )
                        self.game.unmute()
                    except Exception as data:
                        if not self.game.catchExceptions: raise data
                        self.game._agentCrash(agentIndex)
                        self.game.unmute()
                        return
            self.game.display.finish()

            return True

In [3]:
env = Environment()

In [4]:
n_episodes = 10
for i_episode in range(1, n_episodes + 1):
    print("Episode: ", i_episode)

    env.reset()

    while not env.done():
        env.step()

    scores = [env.game.state.getScore()]
    wins = [env.game.state.isWin()]
    winRate = wins.count(True)/ float(len(wins))
    print('Average Score:', sum(scores) / float(len(scores)))
    print('Scores:       ', ', '.join([str(score) for score in scores]))
    print('Win Rate:      %d/%d (%.2f)' % (wins.count(True), len(wins), winRate))
    print('Record:       ', ', '.join([ ['Loss', 'Win'][int(w)] for w in wins]))

    print()

Episode:  1
Average Score: 165.0
Scores:        165.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  2
Average Score: -401.0
Scores:        -401.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  3
Average Score: 131.0
Scores:        131.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  4
Average Score: 489.0
Scores:        489.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  5
Average Score: -428.0
Scores:        -428.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  6
Average Score: -33.0
Scores:        -33.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  7
Average Score: -356.0
Scores:        -356.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  8
Average Score: -34.0
Scores:        -34.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  9
Average Score: -410.0
Scores:        -410.0
Win Rate:      0/1 (0.00)
Record:        Loss

Episode:  10
Average Score: -339.0
Scores:        -339.0
Win Rate:      0/1 (0.00)
